# Lab 2:  Electric Field Mapping

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

In [2]:
# Measured Ruler Distance in Pixels
length_pixels = 562.47 - 22.59
dlength_pixels = 5.0
length_cm = 19.0

lcal = length_cm*0.01/length_pixels
dlcal = dlength_pixels/length_pixels*lcal

print ("Ruler calibration: (%0.3e +/- %0.3e) meters/pixel" % (lcal,dlcal))

Ruler calibration: (3.519e-04 +/- 3.259e-06) meters/pixel


In [3]:
# Read in data from csv files
df1 = pd.read_csv("FieldMapping_Part2_2V.csv")
df2 = pd.read_csv("FieldMapping_Part2_10V.csv")

df = df2.copy()

FileNotFoundError: [Errno 2] No such file or directory: 'FieldMapping_Part2_2V.csv'

In [ ]:
# Define uncertainties in X and Y, in pixels
df['dX']=5.0
df['dY']=5.0

# Calculate X and Y positions, in meters, with uncertainties
df['X_m']=df['X']*lcal
df['dX_m']=df['X_m']*(df['dX']/df['X'] + dlcal/lcal)
df['Y_m']=df['Y']*lcal
df['dY_m']=df['Y_m']*(df['dY']/df['Y'] + dlcal/lcal)

df

In [ ]:
# Calculate the absolute positions of Q1 and Q2

X1_pixels = 530.04
dX1_pixels = 5.0
Y1_pixels = 302.07
dY1_pixels = 5.0

X2_pixels = 284.03
dX2_pixels = 5.0
Y2_pixels = 305.96
dY2_pixels = 5.0

X1 = X1_pixels*lcal
dX1 = X1*(dX1_pixels/X1_pixels + dlcal/lcal)
Y1 = Y1_pixels*lcal
dY1 = Y1*(dY1_pixels/Y1_pixels + dlcal/lcal)

X2 = X2_pixels*lcal
dX2 = X2*(dX2_pixels/X2_pixels + dlcal/lcal)
Y2 = Y2_pixels*lcal
dY2 = Y2*(dY2_pixels/Y2_pixels + dlcal/lcal)

print (X1,dX1,Y1,dY1,X2,dX2,Y2,dY2)

In [ ]:
# Calculate r1 and r2, and errors

df['r1'] = np.sqrt((df['X_m']-X1)**2 + (df['Y_m']-Y1)**2)
df['r2'] = np.sqrt((df['X_m']-X2)**2 + (df['Y_m']-Y2)**2)

df['dr1'] = 1/df['r1']*(np.abs((df['X_m']-X1))*(dX1+df['dX_m']) + (np.abs(df['Y_m']-Y1))*(dY1+df['dY_m']))
df['dr2'] = 1/df['r2']*(np.abs((df['X_m']-X2))*(dX1+df['dX_m']) + (np.abs(df['Y_m']-Y2))*(dY2+df['dY_m']))

df['ID'] = 1.0/df['r1']-1.0/df['r2']
df['dID'] = 1/df['r1']**2*df['dr1'] + 1/df['r2']**2*df['dr2']

df

In [ ]:
# Plot the inverse difference, 1/r1-1/r2, for all data points that we have
plt.figure(figsize=(8, 6), dpi=80)
plt.errorbar(df.index, df['ID'], xerr=0, yerr=df['dID'], fmt='o', capsize=2, capthick=1)

In [ ]:
# Calculate the weighted average of the inverse difference values.
def w_avg(df, values, weights):
    d = df[values]
    w = 1.0/df[weights]**2
    return (d * w).sum()/w.sum(), np.sqrt(1/w.sum())

average, daverage = w_avg(df, 'ID', 'dID')
print (average,daverage)

plt.figure(figsize=(8, 6), dpi=80)
plt.errorbar(df.index, df['ID'], xerr=0, yerr=df['dID'], fmt='o', capsize=2, capthick=1)
npts=len(df)
plt.errorbar(npts+1, average, xerr=0, yerr=daverage, fmt='o', capsize=2, capthick=1)

In [ ]:
# Now, just do the same procedure for the other voltage values!

# As you complete each voltage line analysis, add the extracted averages to the
# csv file called FieldMapping_Averages.csv

In [ ]:
dfavg = pd.read_csv("FieldMapping_Averages.csv")

df = dfavg.copy()
df

In [ ]:
xi = df['Average']
yi = df['Voltage']
sigmaxi = df['dAverage']
sigmayi = df['dVoltage']

##############################

from scipy.odr import *

def fitfunction(B, x):
    '''Linear function y = m*x + b'''
    # B is a vector of the parameters.
    # x is an array of the current x values.
    # x is in the same format as the x passed to Data or RealData.
    #
    # Return an array in the same format as y passed to Data or RealData.
    return B[0]*x + B[1]

linear = Model(fitfunction) # create a Model object based on the fitfuncion we have defined
mydata = RealData(xi, yi, sx=sigmaxi, sy=sigmayi) # create a data object based on our data, include errors.
myodr = ODR(mydata, linear, beta0=[1., 2.]) # create a fitting object, based on the data, fit Model, and an intial set of parameters.
myoutput = myodr.run()  # run the fitting process to get optimized parameters!

myoutput.pprint() # print out the result of the fit

# Now assign the important fit results to some more convenient variables.

popt = myoutput.beta # the vector of optimized parameters
pcov = myoutput.cov_beta # the covariance matrix
perr = myoutput.sd_beta # the vector of ERRORS in the optimized parameters

# The following lines generate upper and lower 99% "Confidence Bands" on the fit, for visualization
# purposes.

ps = np.random.multivariate_normal(popt,pcov,10000)
ysample=np.asarray([fitfunction(pi,xi) for pi in ps])

lower = np.percentile(ysample,16.0,axis=0)
upper = np.percentile(ysample,84.0,axis=0)
middle = (lower+upper)/2.0

print()
print ("Final Result: Y = (%0.9f +/- %0.9f) X + (%0.9f +/- %0.9f)" % (popt[0],perr[0],popt[1],perr[1]))

plt.figure(figsize=(8, 6), dpi=80)

plt.errorbar(xi, yi, xerr=sigmaxi, yerr=sigmayi, fmt='o', capsize=2, capthick=1)

plt.plot(xi,middle)
plt.plot(xi,lower)
plt.plot(xi,upper)

#########################

plt.xlabel('1/R1-1/R2 (meters$^-1$)')
plt.ylabel('Voltage (Volts)')
plt.title('Two Points')
plt.show()